In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

In [5]:
df_train = pd.read_csv('../datasets/train.csv')
df_test = pd.read_csv('../datasets/test.csv')


In [6]:
def convert_passengerID_2_index(df: pd.DataFrame):

    df = df.set_index("PassengerId")
    return df

In [7]:
df_train_cleansed = (
    df_train.pipe(convert_passengerID_2_index)
)

In [17]:
def split_data_2_x_y(df:pd.DataFrame):

    columns = df.columns 
    X_columns = [i for i in columns if i != 'Transported']
    y_column = 'Transported' if 'Transported' in columns else None 

    if y_column:
        return df[X_columns], df[y_column]
    else:
        return df[X_columns], None

In [9]:
X_train, y_train = split_data_2_x_y(df_train_cleansed)

In [13]:
df_test_cleansed = (
    df_test.pipe(convert_passengerID_2_index)
)

In [18]:
X_test, _ = split_data_2_x_y(df_test_cleansed)

In [12]:
X_train.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
PassengerId,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines


In [22]:
ran_clf = RandomForestClassifier(random_state=42)
ran_clf.fit(X_train, y_train)
# ran_clf.predict(X_test)


ValueError: could not convert string to float: 'Europa'